In [1]:
cd ..

c:\Users\jeong\Desktop\person-reid-study\practices\cvae


In [2]:
import pytorch_lightning as pl
import torchvision.transforms
from pl_bolts.callbacks import TensorboardGenerativeModelImageSampler
from pl_bolts.datamodules import MNISTDataModule
from pl_examples import _DATASETS_PATH

from callbacks import CVAEImageSampler, LatentDimInterpolator
from utils import set_persistent_workers

set_persistent_workers(MNISTDataModule)

In [3]:
transforms = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.5, ), (0.5, )),
])

dm = MNISTDataModule(_DATASETS_PATH, num_workers=6,
                     batch_size=64, shuffle=True, drop_last=True,
                     train_transforms=transforms,
                     val_transforms=transforms,
                     test_transforms=transforms)
callbacks = [
    TensorboardGenerativeModelImageSampler(10),
    # LatentDimInterpolator(),
    CVAEImageSampler()
]

In [4]:
from models.cvae import CVAE
model = CVAE(latent_dim=8, batch_norm=False, lr=1e-2)
model

CVAE(
  (encoder): Encoder(
    (embedding): Embedding(10, 10)
    (fc): Sequential(
      (0): Linear(in_features=794, out_features=512, bias=True)
      (1): ReLU()
      (2): Linear(in_features=512, out_features=256, bias=True)
      (3): ReLU()
      (4): Linear(in_features=256, out_features=128, bias=True)
      (5): ReLU()
      (6): Linear(in_features=128, out_features=16, bias=True)
    )
  )
  (decoder): Decoder(
    (embedding): Embedding(10, 10)
    (fc): Sequential(
      (0): Linear(in_features=18, out_features=128, bias=True)
      (1): ReLU()
      (2): Linear(in_features=128, out_features=256, bias=True)
      (3): ReLU()
      (4): Linear(in_features=256, out_features=512, bias=True)
      (5): ReLU()
      (6): Linear(in_features=512, out_features=784, bias=True)
      (7): Sigmoid()
      (8): Unflatten(dim=1, unflattened_size=(1, 28, 28))
    )
  )
)

In [5]:
trainer = pl.Trainer(
    progress_bar_refresh_rate=5,
    max_epochs=1500,
    gpus=-1,
    callbacks=callbacks
)

trainer.fit(model, datamodule=dm)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
C:\Users\jeong\anaconda3\envs\pytorch\lib\site-packages\pytorch_lightning\trainer\configuration_validator.py:99: UserWarning: you passed in a val_dataloader but have no validation_step. Skipping val loop
  rank_zero_warn(f"you passed in a {loader_name} but have no {step_name}. Skipping {stage} loop")
C:\Users\jeong\anaconda3\envs\pytorch\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], co

Epoch 0:   0%|          | 0/750 [00:00<?, ?it/s] 

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.